In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np

In [ ]:
class CommonFeatureExtractor(nn.Module):
    def __init__(self, num_layers=2, embedding_size=384, num_heads=4, dropout_rate=0.2):
        super(CommonFeatureExtractor, self).__init__()

        self.embedding_size = embedding_size

        # Forward Transformer Encoder
        self.forward_encoder_layer = nn.TransformerEncoderLayer(d_model=embedding_size, nhead=num_heads, dropout=dropout_rate)
        self.forward_transformer_encoder = nn.TransformerEncoder(self.forward_encoder_layer, num_layers=num_layers)

        # Backward Transformer Encoder
        self.backward_encoder_layer = nn.TransformerEncoderLayer(d_model=embedding_size, nhead=num_heads, dropout=dropout_rate)
        self.backward_transformer_encoder = nn.TransformerEncoder(self.backward_encoder_layer, num_layers=num_layers)

    def forward(self, x):
        # Permute to (sequence_length, batch_size, embedding_size)
        x = x.permute(1, 0, 2)

        # Forward pass
        forward_output = self.forward_transformer_encoder(x)

        # Backward pass
        backward_output = self.backward_transformer_encoder(torch.flip(x, [0]))

        # Concatenate forward and backward outputs along the feature dimension
        concatenated_output = torch.cat([forward_output, torch.flip(backward_output, [0])], dim=-1)

        # Permute back to (batch_size, sequence_length, embedding_size * 2)
        concatenated_output = concatenated_output.permute(1, 0, 2)

        return concatenated_output


class DomainSpecificFeatureExtractor(nn.Module):
    def __init__(self, num_layers=2, embedding_size=384, num_heads=4, dropout_rate=0.2):
        super(DomainSpecificFeatureExtractor, self).__init__()

        self.embedding_size = embedding_size

        # Forward Transformer Encoder
        self.forward_encoder_layer = nn.TransformerEncoderLayer(d_model=embedding_size, nhead=num_heads, dropout=dropout_rate)
        self.forward_transformer_encoder = nn.TransformerEncoder(self.forward_encoder_layer, num_layers=num_layers)

        # Backward Transformer Encoder
        self.backward_encoder_layer = nn.TransformerEncoderLayer(d_model=embedding_size, nhead=num_heads, dropout=dropout_rate)
        self.backward_transformer_encoder = nn.TransformerEncoder(self.backward_encoder_layer, num_layers=num_layers)

    def forward(self, x):
        # Permute to (sequence_length, batch_size, embedding_size)
        x = x.permute(1, 0, 2)

        # Forward pass
        forward_output = self.forward_transformer_encoder(x)

        # Backward pass
        backward_output = self.backward_transformer_encoder(torch.flip(x, [0]))

        # Concatenate forward and backward outputs along the feature dimension
        concatenated_output = torch.cat([forward_output, torch.flip(backward_output, [0])], dim=-1)

        # Permute back to (batch_size, sequence_length, embedding_size * 2)
        concatenated_output = concatenated_output.permute(1, 0, 2)

        return concatenated_output


In [ ]:
class DomainSpecificClassifier(nn.Module):
    def __init__(self, input_size, num_classes):
        super(DomainSpecificClassifier, self).__init__()
        self.fc = nn.Linear(input_size, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        return self.softmax(self.fc(x))

In [ ]:
class CADA(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes, num_domains):
        super(CADA, self).__init__()
        self.common_feature_extractor = CommonFeatureExtractor(input_size, hidden_size)
        self.domain_specific_feature_extractors = nn.ModuleList([DomainSpecificFeatureExtractor(input_size, hidden_size) for _ in range(num_domains)])
        self.domain_specific_classifiers = nn.ModuleList([DomainSpecificClassifier(hidden_size, num_classes) for _ in range(num_domains)])

    def forward(self, x_sources, x_target):
        common_features = self.common_feature_extractor(x_sources)  # Common feature extraction for source domains
        domain_specific_features = [domain_specific_feature_extractor(x_sources) for domain_specific_feature_extractor in self.domain_specific_feature_extractors]  # Domain-specific feature extraction for each source domain
        domain_outputs = [domain_specific_classifier(domain_specific_feature) for domain_specific_feature, domain_specific_classifier in zip(domain_specific_features, self.domain_specific_classifiers)]  # Domain-specific classification for each source domain
        # Domain alignment and classification for target domain (similar to source domains)
        target_common_features = self.common_feature_extractor(x_target)
        target_domain_specific_features = [domain_specific_feature_extractor(x_target) for domain_specific_feature_extractor in self.domain_specific_feature_extractors]
        target_domain_outputs = [domain_specific_classifier(domain_specific_feature) for domain_specific_feature, domain_specific_classifier in zip(target_domain_specific_features, self.domain_specific_classifiers)]
        return domain_outputs, target_domain_outputs

def MMD_loss(source_features, target_features):
    # Calculate MMD loss
    # This function calculates the Maximum Mean Discrepancy (MMD) loss between source and target features
    return torch.mean(torch.pow(torch.mean(source_features, dim=0) - torch.mean(target_features, dim=0), 2))

def classifier_discrepancy_loss(domain_outputs):
    # Calculate classifier discrepancy loss
    # This function calculates the discrepancy loss between domain-specific classifiers
    num_domains = len(domain_outputs)
    loss = 0
    for i in range(num_domains - 1):
        for j in range(i + 1, num_domains):
            loss += torch.mean(torch.abs(domain_outputs[i] - domain_outputs[j]))
    return loss / (num_domains * (num_domains - 1))


In [ ]:
# 训练函数
def train(model, source_loaders, target_loader, optimizer, num_epochs, alpha, beta):
    model.train()
    num_domains = len(source_loaders)
    for epoch in range(num_epochs):
        for domains_data in zip(*source_loaders, target_loader):
            optimizer.zero_grad()
            source_domain_outputs = []
            for source_loader in source_loaders:
                x_sources, _ = next(iter(source_loader))  # 从每个源领域加载器中获取数据
                source_domain_outputs.append(model.common_feature_extractor(x_sources))  # 提取每个源领域的特征
            target_data, _ = next(iter(target_loader))  # 从目标领域加载器中获取数据
            target_domain_outputs = model.common_feature_extractor(target_data)  # 提取目标领域的特征
            domain_outputs = source_domain_outputs + [target_domain_outputs]  # 汇总所有领域的特征
            mmd_loss = MMD_loss(torch.cat([domain_output for domain_output in domain_outputs[:-1]]), domain_outputs[-1])  # 计算 MMD 损失
            disc_loss = classifier_discrepancy_loss(domain_outputs)  # 计算分类器差异损失
            total_loss = sum([domain_output_loss for domain_output_loss in domain_outputs[:-1]]) + alpha * mmd_loss + beta * disc_loss  # 总损失
            total_loss.backward()
            optimizer.step()

In [ ]:
# 测试函数
def test(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs, _ = model(inputs, None)  # 注意：这里传入的目标数据为 None，因为在测试阶段我们只关注源域数据的输出
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print('Test Accuracy: {:.2f}%'.format(accuracy))

In [ ]:
# 数据集相关参数
input_size = 384  # 输入特征的大小
hidden_size = 1280  # 隐藏层大小
num_classes = 2  # 类别数量
num_domains = 2  # 源域数量
batch_size = 128  # 批量大小


num_layers = 2  # Transformer 层数
num_heads = 4  # Transformer 注意力头数
dropout_rate = 0.2  # Dropout 概率

# 优化器相关参数
learning_rate = 0.001  # 学习率
alpha = 0.1  # MMD 损失的权重
beta = 0.1  # 分类器差异损失的权重

num_epochs = 10  # 训练轮数

In [ ]:
train_portion = 0.2

In [ ]:
D1_normal_logs, D1_abnormal_logs = load_data('BGL', portion=train_portion)
D2_normal_logs, D2_abnormal_logs = load_data('Spirit', portion=train_portion)
D3_normal_logs, D3_abnormal_logs = load_data('TB', portion=train_portion)

# 创建数据集
dataset_A = LogDataset(D1_normal_logs, D1_abnormal_logs)
dataset_B = LogDataset(D2_normal_logs, D2_abnormal_logs)
dataset_C = LogDataset(D3_normal_logs, D3_abnormal_logs)

In [ ]:
train_size_A = int(train_portion * len(dataset_A))
test_size_A = len(dataset_A) - train_size_A
train_dataset_A, test_dataset_A = random_split(dataset_A, [train_size_A, test_size_A])

In [ ]:
train_size_B = int(train_portion * len(dataset_Bb))
test_size_B = len(dataset_B) - train_size_B
train_dataset_B, test_dataset_B = random_split(dataset_B, [train_size_B, test_size_B])

In [ ]:
train_size_C = int(train_portion * len(dataset_C))
test_size_C = len(dataset_C) - train_size_C
train_dataset_C, test_dataset_C = random_split(dataset_C, [train_size_C, test_size_C])

In [ ]:
source_loaders = [DataLoader(dataset_A, batch_size=batch_size, shuffle=True),
                  DataLoader(dataset_B, batch_size=batch_size, shuffle=True)]

target_loader = DataLoader(test_dataset_C, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset_C, batch_size=batch_size, shuffle=False)


In [ ]:
model = CADA(input_size, hidden_size, num_classes, num_domains)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 训练模型
train(model, source_loader, target_loader, optimizer, num_epochs, alpha, beta)

# 在测试集上评估模型
test(model, test_loader)